### Download PyTorch with MNIST Example

Have a look at http://pytorch.org/


Since we're on 3.5, and have no ```cuda```: 

```
pip install http://download.pytorch.org/whl/cu75/torch-0.1.12.post2-cp35-cp35m-linux_x86_64.whl  # 348Mb
pip install torchvision
```

*But* : 
*  https://discuss.pytorch.org/t/blas-performance-on-macos-vs-linux-vs-lua-torch/744
*  https://discuss.pytorch.org/t/solved-archlinux-using-variable-backwards-appears-to-hang-program-indefinitely/1675

#### PyTorch from source
So, let's try to install from source...

```
sudo dnf install cmake

export NO_CUDA=1
git clone https://github.com/pytorch/pytorch.git
cd pytorch
. ~/env3/bin/activate  # Enter into the right virtualenv
python setup.py install
```

Actually, that seemed to work on my AMD home machine (Fedora 25, running Python 3.5.3 in a virtualenv).

It leaves 39.8Mb of files in ```env3/lib64/python3.5/site-packages/torch```.

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.autograd import Variable

In [2]:
random_seed = 42
batch_size = 32

In [3]:
torch.manual_seed(random_seed)
if False:  # GPU...
    torch.cuda.manual_seed(random_seed)

In [4]:
mnist_data_path = './data'

train_loader = torch.utils.data.DataLoader(
    datasets.MNIST(mnist_data_path, train=True, download=True,
                   transform=transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize((0.1307,), (0.3081,))
                   ])),
    batch_size=batch_size, shuffle=True)
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST(mnist_data_path, train=False, transform=transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize((0.1307,), (0.3081,))
                   ])),
    batch_size=batch_size, shuffle=True)

In [5]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 10, kernel_size=5)
        self.conv2 = nn.Conv2d(10, 20, kernel_size=5)
        self.conv2_drop = nn.Dropout2d()
        self.fc1 = nn.Linear(320, 50)
        self.fc2 = nn.Linear(50, 10)

    def forward(self, x):
        x = F.relu(F.max_pool2d(self.conv1(x), 2))
        x = F.relu(F.max_pool2d(self.conv2_drop(self.conv2(x)), 2))
        x = x.view(-1, 320)
        x = F.relu(self.fc1(x))
        x = F.dropout(x, training=self.training)
        x = self.fc2(x)
        return F.log_softmax(x)

model = Net()